<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/5_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

def process_group(group, window_size=3, min_ones=2):
    data = group['col3']
    col4 = group['col4']

    # 存储结果的DataFrame
    results = []

    # 遍历数据
    for i in range(len(data) - window_size + 1):
        window = data.iloc[i:i + window_size]
        count_ones = window.sum()
        if count_ones >= min_ones:
            indices = window[window == 1].index.tolist()
            ones_values = window.iloc[indices].tolist()
            col4_values = col4.iloc[indices].tolist()

            # 创建单行结果
            row = {
                'col1': group['col1'].iloc[0],  # 需要传递原始分组键
                'col2': group['col2'].iloc[0],
                'proportion': count_ones / window_size
            }
            # 添加col3和col4的值
            for j in range(3):  # 固定三列
                col_name_col3 = f'col3_value_{j+1}'
                col_name_col4 = f'col4_value_{j+1}'
                if j < len(ones_values):
                    row[col_name_col3] = ones_values[j]
                    row[col_name_col4] = col4_values[j]
                else:
                    row[col_name_col3] = np.nan
                    row[col_name_col4] = np.nan
            results.append(row)

    return pd.DataFrame(results)

# 示例数据
data = {
    'col1': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B'],
    'col2': ['aa', 'aa', 'aa', 'aa', 'aa', 'bb', 'bb', 'bb', 'bb', 'bb'],
    'col3': [0, 1, 1, 1, 0, 0, 1, 0, 1, 1],
    'col4': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}
df = pd.DataFrame(data)

# 应用函数
result_df = df.groupby(['col1', 'col2']).apply(process_group)

print(result_df.reset_index(drop=True))


先收集所有数据到一个列表中，然后只在最后一次性创建 DataFrame，这样避免了多次创建 DataFrame 的开销。

In [ ]:
import pandas as pd
import numpy as np

def process_group(group, window_size=3, min_ones=2):
    data = group['col3'].reset_index(drop=True)
    col4 = group['col4'].reset_index(drop=True)

    # 存储结果
    results = []

    # 遍历数据以创建结果
    for i in range(len(data) - window_size + 1):
        window = data[i:i + window_size]
        count_ones = window.sum()

        if count_ones >= min_ones:
            indices = window[window == 1].index.tolist()
            ones_values = window.iloc[indices].tolist()
            col4_values = col4.iloc[indices].tolist()

            # 每个窗口结果作为列表存储
            row = [
                group['col1'].iloc[0],  # 分组键
                group['col2'].iloc[0],
                count_ones / window_size
            ]
            # 分别处理每个可能的列
            for j in range(3):  # 固定三列
                if j < len(ones_values):
                    row.append(ones_values[j])
                    row.append(col4_values[j])
                else:
                    row.append(np.nan)
                    row.append(np.nan)
            results.append(row)

    # 一次性创建DataFrame
    if results:
        columns = ['col1', 'col2', 'proportion']
        for j in range(3):
            columns.extend([f'col3_value_{j+1}', f'col4_value_{j+1}'])
        return pd.DataFrame(results, columns=columns)
    else:
        return pd.DataFrame(columns=['col1', 'col2', 'proportion'] + [f'col3_value_{i+1}' for i in range(3)] + [f'col4_value_{i+1}' for i in range(3)])

# 示例数据
data = {
    'col1': ['A', 'A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'B'],
    'col2': ['aa', 'aa', 'aa', 'aa', 'aa', 'bb', 'bb', 'bb', 'bb', 'bb'],
    'col3': [0, 1, 1, 1, 0, 0, 1, 0, 1, 1],
    'col4': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
}
df = pd.DataFrame(data)

# 应用函数
result_df = df.groupby(['col1', 'col2']).apply(process_group).reset_index(drop=True)

print(result_df)
